# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
load_dotenv("OpenAI.env")
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [17]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [27]:
import pandas as pd

# Definition of the tables with their details
data = {
    'table': ['employees', 'salary', 'studies'],  # Table names
    'definition': [
        'Employee information such as name, age, job title, employee ID',
        'Salary details for each year, employee ID, salary amount, year',
        'Educational studies such as institution name, study type, level'
    ]  # Table definitions
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Print the DataFrame to display the tables and their definitions
print(df)



       table                                         definition
0  employees  Employee information such as name, age, job ti...
1     salary  Salary details for each year, employee ID, sal...
2    studies  Educational studies such as institution name, ...


In [29]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [31]:
print(text_tables)

employees: Employee information such as name, age, job title, employee ID
salary: Salary details for each year, employee ID, salary amount, year
studies: Educational studies such as institution name, study type, level


In [33]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [47]:
                                # Creating the prompt with the user question and the table definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What is the salary of the employee in 2021?")


In [49]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary"]
}


In [53]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What is the average salary of employees in 2020?")



In [55]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "salary"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [59]:
pqt1 = prompt_question_tables.format(tables=text_tables, question="What is the salary of the employee in 2021?")
print(return_OAI(pqt1))



{
    "tables": ["employees", "salary"]
}


In [61]:
pqt2 = prompt_question_tables.format(tables=text_tables, question="What is the average salary of employees in 2020?")
print(return_OAI(pqt2))


{
    "tables": ["employees", "salary"]
}


In [63]:
pqt3 = prompt_question_tables.format(tables=text_tables, question="Which employees studied computer science?")
print(return_OAI(pqt3))


```json
{
    "tables": ["employees", "studies"]
}
```
